In [1]:
-- Stored procedure that trains and generates a Python model using the rental_data and a linear regression algorithm
DROP PROCEDURE IF EXISTS generate_rental_py_model;
go
CREATE PROCEDURE generate_rental_py_model (@trained_model varbinary(max) OUTPUT)
AS
BEGIN
    EXECUTE sp_execute_external_script
      @language = N'Python'
    , @script = N'
from sklearn.linear_model import LinearRegression
import pickle

df = rental_train_data

# Get all the columns from the dataframe.
columns = df.columns.tolist()

# Store the variable well be predicting on.
target = "RentalCount"

# Initialize the model class.
lin_model = LinearRegression()

# Fit the model to the training data.
lin_model.fit(df[columns], df[target])

# Before saving the model to the DB table, convert it to a binary object
trained_model = pickle.dumps(lin_model)'

, @input_data_1 = N'select "RentalCount", "Year", "Month", "Day", "WeekDay", "Snow", "Holiday" from dbo.rental_data where Year < 2015'
, @input_data_1_name = N'rental_train_data'
, @params = N'@trained_model varbinary(max) OUTPUT'
, @trained_model = @trained_model OUTPUT;
END;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.048

In [2]:
USE TutorialDB;
DROP TABLE IF EXISTS dbo.rental_py_models;
GO
CREATE TABLE dbo.rental_py_models (
    model_name VARCHAR(30) NOT NULL DEFAULT('default model') PRIMARY KEY,
    model VARBINARY(MAX) NOT NULL
);
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.561

In [3]:
DECLARE @model VARBINARY(MAX);
EXECUTE generate_rental_py_model @model OUTPUT;

INSERT INTO rental_py_models (model_name, model) VALUES('linear_model', @model);

(1 row affected)

Total execution time: 00:00:04.789

In [4]:
SELECT * FROM rental_py_models

(1 row affected)

Total execution time: 00:00:00.031

model_name,model
linear_model,0x800363736B6C6561726E2E6C696E6561725F6D6F64656C2E626173650A4C696E65617252656772657373696F6E0A7100298171017D710228580D0000006669745F696E7465726365707471038858090000006E6F726D616C697A657104895806000000636F70795F5871058858060000006E5F6A6F627371064E5805000000636F65665F7107636E756D70792E636F72652E6D756C746961727261790A5F7265636F6E7374727563740A7108636E756D70790A6E6461727261790A71094B0085710A430162710B87710C52710D284B014B0785710E636E756D70790A64747970650A710F5802000000663871104B004B01877111527112284B0358010000003C71134E4E4E4AFFFFFFFF4AFFFFFFFF4B0074711462894338000000000000F03FB356147F53C2E1BC025E1A3A2988AC3C9D4CF928BE4E9EBC3B9D90CA4209B43CABDAC943C67AEB3C92B40EC90F0A32BD71157471166258090000005F72657369647565737117636E756D70792E636F72652E6D756C746961727261790A7363616C61720A71186812430804720EFAC562D03A711986711A52711B580500000072616E6B5F711C4B07580900000073696E67756C61725F711D680868094B0085711E680B87711F527120284B014B0785712168128943388D8D95B87D0CAB40332A9EE40EFF6240E97534A6012651401CF45FE96F444140975A9166B163214091B48BC719E71F40FDBEB547A6DD0440712274712362580A000000696E746572636570745F7124681868124308000000000080913D712586712652712758100000005F736B6C6561726E5F76657273696F6E71285806000000302E32302E32712975622E


In [ ]:
DROP PROCEDURE IF EXISTS py_predict_rentalcount;
GO
CREATE PROCEDURE py_predict_rentalcount (@model varchar(100))
AS
BEGIN
    DECLARE @py_model varbinary(max) = (select model from rental_py_models where model_name = @model);

    EXECUTE sp_execute_external_script
                @language = N'Python',
                @script = N'

# Import the scikit-learn function to compute error.
from sklearn.metrics import mean_squared_error
import pickle
import pandas

rental_model = pickle.loads(py_model)

df = rental_score_data

# Get all the columns from the dataframe.
columns = df.columns.tolist()

# Variable you will be predicting on.
target = "RentalCount"

# Generate the predictions for the test set.
lin_predictions = rental_model.predict(df[columns])
print(lin_predictions)

# Compute error between the test predictions and the actual values.
lin_mse = mean_squared_error(lin_predictions, df[target])
#print(lin_mse)

predictions_df = pandas.DataFrame(lin_predictions)

OutputDataSet = pandas.concat([predictions_df, df["RentalCount"], df["Month"], df["Day"], df["WeekDay"], df["Snow"], df["Holiday"], df["Year"]], axis=1)
'
, @input_data_1 = N'Select "RentalCount", "Year" ,"Month", "Day", "WeekDay", "Snow", "Holiday"  from rental_data where Year = 2015'
, @input_data_1_name = N'rental_score_data'
, @params = N'@py_model varbinary(max)'
, @py_model = @py_model
with result sets (("RentalCount_Predicted" float, "RentalCount" float, "Month" float,"Day" float,"WeekDay" float,"Snow" float,"Holiday" float, "Year" float));

END;
GO

In [ ]:
DROP TABLE IF EXISTS [dbo].[py_rental_predictions];
GO

CREATE TABLE [dbo].[py_rental_predictions](
 [RentalCount_Predicted] [int] NULL,
 [RentalCount_Actual] [int] NULL,
 [Month] [int] NULL,
 [Day] [int] NULL,
 [WeekDay] [int] NULL,
 [Snow] [int] NULL,
 [Holiday] [int] NULL,
 [Year] [int] NULL
) ON [PRIMARY]
GO

In [ ]:
--Insert the results of the predictions for test set into a table
INSERT INTO py_rental_predictions
EXEC py_predict_rentalcount 'linear_model';

-- Select contents of the table
SELECT * FROM py_rental_predictions;